> 사실상 python multiprocessing 에서 조금 추가한게 torch.multiprocessing 이다. 큰 차이는 없다. 공식설명 : 

>torch.multiprocessing is a drop in replacement for Python’s multiprocessing module. It supports the exact same operations, but extends it, so that all tensors sent through a multiprocessing.Queue, will have their data moved into shared memory and will only send a handle to another process.

> process간 자원공유는 이루어지지 않는다. multiprocessing에서는... 그렇기에 torch.distributed 를 통해서 process간 통신이 이루어지게 한다. torch.distributed는 멀티프로세스에 대해서, 멀티프로세스간 통신 및 취합이 가능하게 해준다. 

> 그렇기에 torch.nn.parallel.ddp 처럼 멀티프로세스로 돌아가는 것의 경우에는 멀티프로세스 생성하고 프로세스간 통신을 하고 gradient을 all-reduce 방식으로 취합할 수 있도록 해주는것이다. init_process_group을 통해 process간 통신을 master에서 관리하도록 하는것. 

> ddp의 경우는 원래는 mulit 머신, 즉 여러개의 컴퓨터 통신때문에 고안된것. dp는 이게 불가능하다. 




In [1]:
import numpy as np
import torch
from torch.multiprocessing import Pool

X = np.array([[1, 3, 2, 3], [2, 3, 5, 6], [1, 2, 3, 4]])
X = torch.DoubleTensor(X).cuda()

def X_power_func(j):
    X_power = X**j
    return X_power

if __name__ == '__main__':
  with Pool(processes = 2) as p:   # Paralleizing over 2 GPUs
    results = p.map(X_power_func, range(8))

results

/home/irteam/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: CUDA error: initialization error
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [9]:
  
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
from torch.nn.parallel import DistributedDataParallel as DDP
import os

"""
간단한 모델로의 예시
world_size는 사용가능한 GPU의 갯수를 의미한다.
"""
os.environ['MASTER_ADDR'] = '0.0.0.0'
os.environ['MASTER_PORT'] = '8892'
device = torch.device("cuda")


def example(rank, world_size):
  #default process group 을 만듬
  dist.init_process_group("nccl", rank = rank, world_size = world_size)
  
  #간단한 모델
  model = nn.Linear(10, 10)
  
  #DDP model 만들기
  ddp_model = DDP(model)
  
  loss_fn = nn.MSELoss()
  optimizer = optim.SGD(ddp_model.parameters(), lr = 0.001)
  
  #순전파
  
  for i in range(1500):

    outputs = ddp_model(torch.randn(20, 10).to(device))
    labels = torch.randn(20, 10).to(device)
  
    #역전파
    loss_fn(outputs, labels).backward()
  
    #parameter 업데이트
    optimizer.step()
    print('one step')



In [10]:

world_size = 2
mp.spawn(example, args = (world_size, ), nprocs = world_size, join = False)



Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/irteam/.pyenv/versions/3.7.5/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/irteam/.pyenv/versions/3.7.5/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'example' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/irteam/.pyenv/versions/3.7.5/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/irteam/.pyenv/versions/3.7.5/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'example' on <module '__main__' (built-in)>
